# Machine Learning: Assignment #2

Bárbara Nóbrega Galiza – 202408654​

Cláudia Oliveira - 202005668​

Marta Campos - 202008344

In [1]:
import pandas as pd

### Data Preprocessing

Datasets

In [2]:
# Diagnoses and their descriptions
diag = pd.read_csv('tables/DIAGNOSES_ICD.csv')
desc = pd.read_csv('tables/D_ICD_DIAGNOSES.csv')

# Patient demographic information
patients = pd.read_csv('tables/PATIENTS.csv')

# Hospital admissions 
admissions = pd.read_csv('tables/ADMISSIONS.csv')

# ICU stays per patient admission
icustays = pd.read_csv('tables/ICUSTAYS.csv')

# Charted events: vital signs, lab results, nurse observations, etc.
#chartevents = pd.read_csv('tables/CHARTEVENTS.csv', low_memory=False)

# Inputs: fluids, medications, nutrition via mechanical ventilation
#inputevents = pd.read_csv('tables/INPUTEVENTS_MV.csv', low_memory=False)

# Dictionary that maps ITEMIDs to their names/descriptions
items = pd.read_csv('tables/D_ITEMS.csv')

Choosing the Disease

In [6]:
d = diag[diag['ICD9_CODE'] == '4019']
#print(d.head(15))
#d['SUBJECT_ID'].value_counts()
print(d['SUBJECT_ID'].nunique())

17613


In [ ]:
print(f"Columns of Diagnosis: {diag.columns} \n")
print(f"Columns of Description of Diseases: {desc.columns} \n")

top_codes = diag['ICD9_CODE'].value_counts().head(15).reset_index()
top_codes.columns = ['ICD9_CODE', 'count']

top_diseases = top_codes.merge(desc[['ICD9_CODE', 'SHORT_TITLE']], on='ICD9_CODE', how='left')

print(top_diseases)

Columns of Diagnosis: Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object') 

Columns of Description of Diseases: Index(['ROW_ID', 'ICD9_CODE', 'SHORT_TITLE', 'LONG_TITLE'], dtype='object') 

   ICD9_CODE  count               SHORT_TITLE
0       4019  20703          Hypertension NOS
1       4280  13111                   CHF NOS
2      42731  12891       Atrial fibrillation
3      41401  12429  Crnry athrscl natve vssl
4       5849   9119  Acute kidney failure NOS
5      25000   9058  DMII wo cmp nt st uncntr
6       2724   8690    Hyperlipidemia NEC/NOS
7      51881   7497  Acute respiratry failure
8       5990   6555  Urin tract infection NOS
9      53081   6326         Esophageal reflux
10      2720   5930   Pure hypercholesterolem
11      V053   5779  Need prphyl vc vrl hepat
12      V290   5519    NB obsrv suspct infect
13      2859   5406                Anemia NOS
14      2449   4917        Hypothyroidism NOS


In [7]:
diag.head(15)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
5,1302,109,172335,6.0,2762
6,1303,109,172335,7.0,7100
7,1304,109,172335,8.0,2767
8,1305,109,172335,9.0,7243
9,1306,109,172335,10.0,45829


In [8]:
icustays.head(15)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
5,370,273,158689,241507,carevue,MICU,MICU,52,52,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862
6,371,274,130546,254851,carevue,MICU,MICU,12,12,2114-06-28 22:28:44,2114-07-07 18:01:16,8.8143
7,372,275,129886,219649,carevue,CCU,CCU,7,7,2170-10-07 11:28:53,2170-10-14 14:38:07,7.1314
8,373,276,135156,206327,carevue,CCU,CCU,57,57,2147-11-20 09:02:23,2147-11-21 17:08:52,1.3378
9,374,277,171601,272866,carevue,NICU,NICU,56,56,2132-10-21 21:11:46,2132-10-22 14:44:48,0.7313


In [11]:
icu_diag = icustays.merge(diag, on="HADM_ID", how="inner")
diag_counts = icu_diag.groupby("ICUSTAY_ID")["ICD9_CODE"].nunique().reset_index()
diag_counts = diag_counts.rename(columns={"ICD9_CODE": "num_diagnoses"})
has_multiple = diag_counts["num_diagnoses"] > 1
print(f"ICU stays with multiple diagnoses: {has_multiple.sum()} out of {len(diag_counts)}")
diag_counts

ICU stays with multiple diagnoses: 61215 out of 61532


,ICUSTAY_ID,num_diagnoses
0,200001,20
1,200003,6
2,200006,8
3,200007,7
4,200009,6
...,...,...
61527,299993,2
61528,299994,19
61529,299995,5
61530,299998,13


### Data Preparation

### Training and Validation

### Analysis of Results and Interpretation